In [1]:
#importing libraries
import pandas as pd
import os
import config as cn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [3]:
# Read in data and set p value Elimination
df = pd.read_csv(os.path.join(cn.clean_dir,'final_aggregated_categories.csv'), dtype = 'unicode')
player_cols =  ['Unnamed: 0','Unnamed: 0.1','player','realm','gear_score','last_login',
            'time_since_login','status']
df = df.drop(player_cols, axis = 1)
dfp = pd.DataFrame()
df = df[df.engagement.astype(float) != 1]
df.head()

,engagement,Alterac Valley,Arathi Basin,Archaeology,Arena,Argent Tournament,Ashran,Battle Dungeon,Battle Raid,Battle for Azeroth,...,Tol Barad,Twin Peaks,Visions of N'Zoth,War Effort,Warsong Gulch,Winter Veil,Wintergrasp,World,World Events,Wrath of the Lich King
0,0.0,3,3,79,6,15,2,0,0,19,...,6,1,0,1,5,5,13,4,1,12
1,0.0,3,2,0,4,0,2,10,3,22,...,0,0,0,3,4,3,5,5,0,11
2,0.0,3,2,0,2,5,2,26,0,23,...,1,0,0,5,2,3,0,5,0,11
3,0.0,3,3,0,2,0,2,1,0,23,...,0,0,0,3,2,3,4,5,0,11
6,0.0,3,2,0,2,12,6,2,0,22,...,0,0,0,3,2,3,8,5,0,11


In [4]:
split = StratifiedShuffleSplit(n_splits = 10, test_size = 0.25, random_state = 17)
for train_index, test_index in split.split(df, df.engagement):
    strat_train = df.iloc[train_index][:]
    strat_test = df.iloc[test_index][:]

y_train = strat_train.engagement
X_train = strat_train.drop('engagement', axis = 1)
y_test = strat_train.engagement
X_test = strat_test.drop('engagement', axis = 1)

In [ ]:
feature_selector = SequentialFeatureSelector(RandomForestClassifier(n_jobs=1),
           k_features=15,
           forward=True,
           verbose=2,
           scoring='roc_auc',
           cv=4)
features = feature_selector.fit(np.array(X_train.fillna(0)), y_train)
filtered_features = train_features.columns[list(features.k_feature_idx_)]
print(filtered_features)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  86 out of  86 | elapsed:  5.8min finished

[2020-06-18 17:46:27] Features: 1/15 -- score: 0.8275319279890125[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  85 out of  85 | elapsed:  7.5min finished

[2020-06-18 17:53:58] Features: 2/15 -- score: 0.8657051075077244[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  84 out of  84 | elapsed:  8.8min finished

[2020-06-18 18:02:47] Features: 3/15 -- score: 0.8656710958239484[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=17, max_depth=3)
clf.fit(X_train[filtered_features], y_train)

In [ ]:
train_pred = clf.predict_proba(X_train[filtered_features])
print('Accuracy on training set: {}'.format(roc_auc_score(X_train, train_pred[:,1])))

test_pred = clf.predict_proba(X_test[filtered_features].fillna(0))
print('Accuracy on test set: {}'.format(roc_auc_score(y_test, test_pred [:,1])))